# Lab 3 - Creating partitioned parquet files

In this lab, we will perform our first round of data preparation by writing the larger files (`XREF` and the yearly `parcel` files to the `parquet` format.  

In the process, we will discuss and investigate an important concept in managing lots of data: the principle of locality.  Big data problem as IO bound, meaning that almost all of the time/resources will be used managing the input/output of data.  The principle of locality holds that is often reused over a short period of time (temporal locality) and data that is stored in similar locations tend to be used at similar points in a program (spatial locality).  The `parquet` always us to partition a data set to leverage these properties.   *The correct partitioning can result in orders of magnitude speed up in processing time!*



## The Principle of Locality.

![image.png](attachment:image.png)

We can leverage the behavior of the operating system (OS)--in particular the loading of chucks of data in proximity and keeping that data in memory for a time--by partitioning our data so that similar data is stored together.

## We need to group the data by lake and distance to the lake

![image.png](attachment:image.png)

## Important notes on data size and platform selection

The primary challenge in big data processing is the size of the data.  In this lab, we will be working with the parcel files, which individually are small enough to fit into memory on a single machine, but become problematic when processed in bulk.  Furthermore, we will be aggregating and filtering the data during feature engineering, so storing the data in a columnar format will be beneficial for performance.

Consequently, we will need to use `polars` support for reading and writing lazy data frames, such as `scan_csv`, `scan_parquet`, and `sink_parquet`, which will allow us to take full advatange of predicate and projection pushdown when processing the data.  This will help to minimize the amount of data that needs to be read and processed, which will improve performance.

In [1]:
!pip install deltalake

In [2]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!mkdir -p data
!unzip "/content/drive/MyDrive/data.zip" -d ./data


Streaming output truncated to the last 5000 lines.
   creating: ./data/data/xref_partitioned.parquet/Monit_MAP_CODE1=10000200-04/distance_category=within%20500%20m/
  inflating: ./data/data/xref_partitioned.parquet/Monit_MAP_CODE1=10000200-04/distance_category=within%20500%20m/0.parquet  
   creating: ./data/data/xref_partitioned.parquet/Monit_MAP_CODE1=10000200-04/distance_category=within_500_meters/
  inflating: ./data/data/xref_partitioned.parquet/Monit_MAP_CODE1=10000200-04/distance_category=within_500_meters/0.parquet  
   creating: ./data/data/xref_partitioned.parquet/Monit_MAP_CODE1=10000500-01/
   creating: ./data/data/xref_partitioned.parquet/Monit_MAP_CODE1=10000500-01/distance_category=between%20501-1600m/
  inflating: ./data/data/xref_partitioned.parquet/Monit_MAP_CODE1=10000500-01/distance_category=between%20501-1600m/0.parquet  
   creating: ./data/data/xref_partitioned.parquet/Monit_MAP_CODE1=10000500-01/distance_category=between_501_1600_meters/
  inflating: ./data/data

In [4]:
!mv data/data/* data/
!rm -r data/data


mv: cannot move 'data/data/__MACOSX' to 'data/__MACOSX': Directory not empty
mv: cannot move 'data/data/MinneMUDAC_raw_files' to 'data/MinneMUDAC_raw_files': Directory not empty
mv: cannot move 'data/data/parcel.delta' to 'data/parcel.delta': Directory not empty
mv: cannot move 'data/data/xref_partitioned.parquet' to 'data/xref_partitioned.parquet': Directory not empty


## Problem 1 - Understanding the big picture and tables keys

**Tasks.**  

1. Explain why, in the case of parcel data, it is important to group the rows by lake id and distance to the lake.
2. Neither of these columns is present in the parcel data files.  How will we go about adding this information?

> <font color="orange"> Your thoughts here </font>

## Problem 2 - Writing the XREF to a partitioned parquet "file"

**Tasks.**  

1. Use `polars` to build a lazy query that performs the following tasks.
   - Scan the `XREF` data and select the relevant columns (Lake ID, centroid lat & long, distance to the lake).  Be sure to set `infer_schema=False` to keep the latitude and longitude columns as strings.<br>
   - Create a new categorical variable named with three categories based on distance to the lake: `within 500m`, `between 501-1600m`, and `over 1600m`.<br>
2. Test your query by collecting the first 10 or so rows using limit and collect.
3. Write the table as a partitioned parquet file (by Lake ID).

In [5]:
!ls -lh data/MinneMUDAC_raw_files/

total 11G
-rw-rw-rw- 1 root root 356M Dec  1 17:18 2002_metro_tax_parcels.txt
-rw-rw-rw- 1 root root 312M Dec  1 17:18 2003_metro_tax_parcels.txt
-rw-rw-rw- 1 root root 664M Dec  1 17:18 2004_metro_tax_parcels.txt
-rw-rw-rw- 1 root root 690M Dec  1 17:18 2005_metro_tax_parcels.txt
-rw-rw-rw- 1 root root 723M Dec  1 17:18 2006_metro_tax_parcels.txt
-rw-rw-rw- 1 root root 738M Dec  1 17:19 2007_metro_tax_parcels.txt
-rw-rw-rw- 1 root root 805M Dec  1 17:19 2008_metro_tax_parcels.txt
-rw-rw-rw- 1 root root 824M Dec  1 17:19 2009_metro_tax_parcels.txt
-rw-rw-rw- 1 root root 795M Dec  1 17:19 2010_metro_tax_parcels.txt
-rw-rw-rw- 1 root root 790M Dec  1 17:20 2011_metro_tax_parcels.txt
-rw-rw-rw- 1 root root 818M Dec  1 17:20 2012_metro_tax_parcels.txt
-rw-rw-rw- 1 root root 829M Dec  1 17:20 2013_metro_tax_parcels.txt
-rw-rw-rw- 1 root root 845M Dec  1 17:21 2014_metro_tax_parcels.txt
-rw-rw-rw- 1 root root 834M Dec  1 17:21 2015_metro_tax_parcels.txt
-rw-rw-rw- 1 root root  12M Dec  1 17:

In [6]:
# Place your code/thoughts in one or more code/markdown cells, respectively.

In [7]:
from glob import glob

glob('data/MinneMUDAC_raw_files/*.txt')

['data/MinneMUDAC_raw_files/mces_lakes_1999_2014.txt',
 'data/MinneMUDAC_raw_files/2015_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2003_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/mces_lakes_1999_2014_v2.txt',
 'data/MinneMUDAC_raw_files/2014_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2009_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/Parcel_Lake_Monitoring_Site_Xref.txt',
 'data/MinneMUDAC_raw_files/2008_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2006_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2007_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2002_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2011_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2013_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2005_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2004_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2010_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2012_metro_tax_parcels.txt']

In [8]:
import polars as pl

(xref := (
    pl.scan_csv(
        'data/MinneMUDAC_raw_files/Parcel_Lake_Monitoring_Site_Xref.txt',
        separator = '\t',
        infer_schema=False
    )
    .select([
        'Monit_MAP_CODE1',
        pl.col('Distance_Parcel_Lake_meters').cast(pl.Float64),
        'centroid_long',
        'centroid_lat',
    ])
    .with_columns([
        pl.when(pl.col('Distance_Parcel_Lake_meters') <= 500)
            .then(pl.lit('within_500_meters'))
            .when((pl.col('Distance_Parcel_Lake_meters') > 500) & (pl.col('Distance_Parcel_Lake_meters') <= 1600))
            .then(pl.lit('between_501_1600_meters'))
            .otherwise(pl.lit('over_1600_meters'))
        .alias('distance_category')
    ])

))
xref.limit(10).collect()

Monit_MAP_CODE1,Distance_Parcel_Lake_meters,centroid_long,centroid_lat,distance_category
str,f64,str,str,str
"""19007900-01""",2571.526792,"""-93.11451""","""44.94283""","""over_1600_meters"""
"""19007900-01""",2515.373802,"""-93.11539""","""44.94234""","""over_1600_meters"""
"""19007900-01""",2511.924959,"""-93.11556""","""44.94231""","""over_1600_meters"""
"""19007900-01""",2502.99164,"""-93.11572""","""44.94223""","""over_1600_meters"""
"""19007900-01""",2465.206234,"""-93.1158""","""44.94189""","""over_1600_meters"""
"""19007900-01""",2631.461256,"""-93.11218""","""44.94325""","""over_1600_meters"""
"""19007900-01""",3074.8658,"""-93.11036""","""44.94711""","""over_1600_meters"""
"""19007900-01""",3074.67095,"""-93.11048""","""44.94712""","""over_1600_meters"""
"""19007900-01""",3040.091456,"""-93.11042""","""44.9468""","""over_1600_meters"""


In [9]:
xref.sink_parquet(
    pl.PartitionByKey('data/xref_partitioned.parquet',
                      by=['Monit_MAP_CODE1','distance_category']),
    mkdir=True
)

### Part 3 - Inspecting the partitioned `parquet` file

**Tasks.** Inspect the resulting "file" (actually a folder) from the last set and answer the following questions.

1. What impact did the partitioning have on the way the data was saved?
2. How would this structure help `polars` apply predicate pushdown?
3. How would this structure provide help via the principle of locality.  
4. When working with a cluster of machines, operations such as `group_by` are WIDE operations, meaning they generally need to shuffle data between machines.  Such a suffle is *very* expensive.  In a future lab, we will be creating features for each labke by grouping and aggregating on the lakes and years.  How would applying a similar structure to the parcel data help in this case? **Hint.** Remember that the data will be distributed across multiple machines using the partitions, i.e., each machine will load all or some of the same partition(s).


> <font color="orange"> Your thoughts here </font>

### Part 4 - Filter parcels and joining lake id - one file

One of our programming principles is "do it to one before you do it to all".  Let's practice this by building a query using the `2004` file. Start by saving the path to the `2004` parcel file to the variable `q`.

**Tasks.**

1. Write a query in `polars` that performs the following tasks.
   - Build a query to scan corresponding CSV,
   - Selects common columns list the script file from the previous lab,
   - Joins on the necessary info from the `XREF` (lake ID, distance to the lake, distance category defined above, and centroid lat & long).
2. Test your query by collecting the first 10 or so rows using limit and collect.

Do this in a single `polars` dot chain.

**Important note.** When using a lazy query with a join, both tables must be lazy.  You can achieve this by using `pl.scan_csv` for the parcel data and `pl.read_parquet(...).lazy()` for the `XREF` data.

In [10]:
from column_data import cols_to_keep

cols_to_keep

['ACRES_DEED',
 'ACRES_POLY',
 'AGPRE_ENRD',
 'AGPRE_EXPD',
 'AG_PRESERV',
 'BASEMENT',
 'BLDG_NUM',
 'BLOCK',
 'CITY',
 'CITY_USPS',
 'COOLING',
 'COUNTY_ID',
 'DWELL_TYPE',
 'EMV_BLDG',
 'EMV_LAND',
 'EMV_TOTAL',
 'FIN_SQ_FT',
 'GARAGE',
 'GARAGESQFT',
 'GREEN_ACRE',
 'HEATING',
 'HOMESTEAD',
 'HOME_STYLE',
 'LANDMARK',
 'LOT',
 'MULTI_USES',
 'NUM_UNITS',
 'OPEN_SPACE',
 'OWNER_MORE',
 'OWNER_NAME',
 'OWN_ADD_L1',
 'OWN_ADD_L2',
 'OWN_ADD_L3',
 'PARC_CODE',
 'PIN',
 'PLAT_NAME',
 'PREFIXTYPE',
 'PREFIX_DIR',
 'SALE_DATE',
 'SALE_VALUE',
 'SCHOOL_DST',
 'SPEC_ASSES',
 'STREETNAME',
 'STREETTYPE',
 'SUFFIX_DIR',
 'Shape_Area',
 'Shape_Leng',
 'TAX_ADD_L1',
 'TAX_ADD_L2',
 'TAX_ADD_L3',
 'TAX_CAPAC',
 'TAX_EXEMPT',
 'TAX_NAME',
 'TOTAL_TAX',
 'UNIT_INFO',
 'USE1_DESC',
 'USE2_DESC',
 'USE3_DESC',
 'USE4_DESC',
 'WSHD_DIST',
 'XUSE1_DESC',
 'XUSE2_DESC',
 'XUSE3_DESC',
 'XUSE4_DESC',
 'YEAR_BUILT',
 'Year',
 'ZIP',
 'ZIP4',
 'centroid_lat',
 'centroid_long']

In [11]:
from column_data import cols_to_keep
import polars as pl
(parcel_2004 :=(
    pl.scan_csv(
        'data/MinneMUDAC_raw_files/2004_metro_tax_parcels.txt',
        separator='|',
        infer_schema=False
)
.select(cols_to_keep)
     .join(
        xref,
        left_on=['centroid_lat','centroid_long'],
        right_on=['centroid_lat','centroid_long'],
        how='inner'
     )
)).limit(10).collect()

ACRES_DEED,ACRES_POLY,AGPRE_ENRD,AGPRE_EXPD,AG_PRESERV,BASEMENT,BLDG_NUM,BLOCK,CITY,CITY_USPS,COOLING,COUNTY_ID,DWELL_TYPE,EMV_BLDG,EMV_LAND,EMV_TOTAL,FIN_SQ_FT,GARAGE,GARAGESQFT,GREEN_ACRE,HEATING,HOMESTEAD,HOME_STYLE,LANDMARK,LOT,MULTI_USES,NUM_UNITS,OPEN_SPACE,OWNER_MORE,OWNER_NAME,OWN_ADD_L1,OWN_ADD_L2,OWN_ADD_L3,PARC_CODE,PIN,PLAT_NAME,PREFIXTYPE,PREFIX_DIR,SALE_DATE,SALE_VALUE,SCHOOL_DST,SPEC_ASSES,STREETNAME,STREETTYPE,SUFFIX_DIR,Shape_Area,Shape_Leng,TAX_ADD_L1,TAX_ADD_L2,TAX_ADD_L3,TAX_CAPAC,TAX_EXEMPT,TAX_NAME,TOTAL_TAX,UNIT_INFO,USE1_DESC,USE2_DESC,USE3_DESC,USE4_DESC,WSHD_DIST,XUSE1_DESC,XUSE2_DESC,XUSE3_DESC,XUSE4_DESC,YEAR_BUILT,Year,ZIP,ZIP4,centroid_lat,centroid_long,Monit_MAP_CODE1,Distance_Parcel_Lake_meters,distance_category
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,str
"""0.0""","""0.21""",null,null,"""N""",null,"""117""","""002""","""WACONIA CITY""","""WACONIA""",null,"""019""",null,"""127900.0""","""41800.0""","""169700.0""","""1124.0""","""Y""",null,"""N""",null,"""Y""","""SPLIT ENTY""",null,"""003""","""N""",null,null,null,null,null,null,null,"""1.0""","""019-752600170""","""FOX RUN 1ST ADDITION""",null,null,"""2004-05-20""","""207000.0""","""0110""","""21.0""","""CARVER""","""SQ""",null,null,null,"""117 CARVER SQ""","""WACONIA, MN 55387-9686""",null,"""0.0""",null,"""JOHNSON,ROBERT C""","""1715.0""",null,"""RESIDENTIAL""",null,null,null,"""CARVER CREEK CWM""",null,null,null,null,"""1982.0""","""2004""","""55387""","""9686""","""44.83933""","""-93.78435""","""10008400-01""",1561.777167,"""between_501_1600_meters"""
"""0.0""","""0.27""",null,null,"""N""",null,"""853""","""002""","""CARVER CITY""","""CARVER""",null,"""019""",null,"""194800.0""","""44600.0""","""239400.0""","""1324.0""","""Y""",null,"""N""",null,"""Y""","""COLONIAL""",null,"""002""","""N""",null,null,null,null,null,null,null,"""1.0""","""019-200890130""","""CARVER CREEK 4TH ADDITION""",null,null,"""2002-06-21""","""223388.0""","""0112""","""21.0""","""GILFILLAN""","""AVE""",null,null,null,"""853 GILFILLAN AVE""","""CARVER, MN 55315-4513""",null,"""0.0""",null,"""SANDERS,MARC A & GLYNIS R""","""3567.0""",null,"""RESIDENTIAL""",null,null,null,"""CARVER CREEK CWM""",null,null,null,null,"""2002.0""","""2004""","""55315""","""4513""","""44.75703""","""-93.63912""","""10003100-01""",3970.523552,"""over_1600_meters"""
"""0.0""","""0.0""",null,null,null,null,"""310""","""030""","""CHAMPLIN""","""CHAMPLIN""",null,"""053""",null,"""103100.0""","""35000.0""","""138100.0""","""0.0""",null,null,"""N""",null,"""Y""",null,null,null,null,null,"""N""",null,"""W A PELTIER & C PELTIER""",null,null,null,"""0.0""","""053-2412022140037""","""COLBURNS ADDN TOWNSITE OF CHAM…",null,null,"""2003-10-01""","""171000.0""","""011""","""0.0""","""GOODRICH AVE N""",null,null,"""893.736207033""","""121.441182114""","""310 GOODRICH AVE""","""CHAMPLIN MN 55316""",null,"""1247.0""","""N""","""WENDY & CORRIE PELTIER""","""1348.0""",null,null,null,null,null,"""0""",null,null,null,null,"""1962.0""","""2004""","""55316""",null,"""45.1906""","""-93.402""","""02008400-01""",4681.771544,"""over_1600_meters"""
"""0.0""","""0.46""",null,null,"""N""",null,"""834""","""002""","""CARVER CITY""","""CARVER""",null,"""019""",null,"""287600.0""","""70600.0""","""358200.0""","""1400.0""","""Y""",null,"""N""",null,"""Y""","""COLONIAL""",null,"""003""","""N""",null,null,null,null,null,null,null,"""1.0""","""019-200800220""","""CARVER BLUFFS SIXTH ADDITION""",null,null,"""2001-09-30""","""360149.0""","""0112""","""21.0""","""RAMSEY""","""AVE""",null,null,null,"""834 RAMSEY AVE""","""CARVER, MN 55315-4515""",null,"""0.0""",null,"""KNUTSEN,JOSEPH K & JENNIFER A""","""5583.0""",null,"""RESIDENTIAL""",null,null,null,"""CARVER CREEK CWM""",null,null,null,nul

### Part 5 - Filter parcels and joining lake id - All files

Next, we will partition and write each of the 2004-2015 parcel files to a `parquet` "file".  To do this, complete each of the following tasks.

**Tasks.**

2. We need to adapt the query from the previous step to work for all the years, which can be accomplished by placing that query in a comprehension.  The query should perform the following tasks for each year from 2004-2015.
   - Scan the corresponding CSV (again, `infer_schema=False` to keep lat & long as strings), <br>
   - Select the common columns (import from `parcel.py`),<br>
   - Join on the necessary info from the `XREF` (lake ID, distance to the lake, distance category defined above, and centroid lat & long).<br>
   - Compute the distance to lake using the great circle distance, then construct the lake distance categorical variable as defined above.<br>
   <br>
   Do this in a single `polars` dot chain.<br>
3. Use a comprehension to create a list of lazy data frames for each year from 2004-2015 that contain the joined parcel data.  The expression should use the query from the previous step.  First, test your query on a limited number of rows (e.g., 1000) to ensure it works correctly.
4. Now we want to sink all of the parcel data to a single partitioned parquet "file".  You will need to
    
**Important note.** When writing the parcel files separately, each parcel files took 10+ minutes on my laptop, so writing the combined file will take a while.  Pick a convenient time and be sure to plug in your laptop!

In [12]:
# Place your code/thoughts in one or more code/markdown cells, respectively.

In [13]:
import re

In [14]:
from glob import glob
(parcel_paths := sorted([
    path for path in glob('data/MinneMUDAC_raw_files/*parcel*.txt')
    if any(str(year) in path for year in range(2004, 2016))
]))

['data/MinneMUDAC_raw_files/2004_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2005_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2006_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2007_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2008_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2009_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2010_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2011_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2012_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2013_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2014_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2015_metro_tax_parcels.txt']

In [15]:
# exclude_pattern = re.compile(r'200[23]')
# (parcel_paths := sorted([
#     path for path in glob('data/MinneMUDAC_raw_files/*parcel*.txt')
#     if not exclude_pattern.search(path)
# ]))

In [16]:
from column_data import cols_to_keep
(parcel_lazy_queries := [
    pl.scan_csv(path, separator='|', infer_schema=False)
    .select(cols_to_keep)
    .join(
        xref,
        left_on=['centroid_lat','centroid_long'],
        right_on=['centroid_lat','centroid_long'],
        how='inner'
    )
    for path in parcel_paths
])

In [17]:
[q.limit(2).collect() for q in parcel_lazy_queries]

[shape: (2, 73)
 ┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
 │ ACRES_DEE ┆ ACRES_POL ┆ AGPRE_ENR ┆ AGPRE_EXP ┆ … ┆ centroid_ ┆ Monit_MAP ┆ Distance_ ┆ distance │
 │ D         ┆ Y         ┆ D         ┆ D         ┆   ┆ long      ┆ _CODE1    ┆ Parcel_La ┆ _categor │
 │ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ke_meters ┆ y        │
 │ str       ┆ str       ┆ str       ┆ str       ┆   ┆ str       ┆ str       ┆ ---       ┆ ---      │
 │           ┆           ┆           ┆           ┆   ┆           ┆           ┆ f64       ┆ str      │
 ╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
 │ 0.0       ┆ 0.21      ┆ null      ┆ null      ┆ … ┆ -93.78435 ┆ 10008400- ┆ 1561.7771 ┆ between_ │
 │           ┆           ┆           ┆           ┆   ┆           ┆ 01        ┆ 67        ┆ 501_1600 │
 │           ┆           ┆           ┆           ┆   ┆           ┆

In [ ]:
for i,q in enumerate(parcel_lazy_queries):
    (q.collect(engine = 'streaming')
     .write_delta(
         './data/parcel.delta',
         mode='overwrite' if i == 0 else 'append',
         delta_write_options={'partition_by' : ['Monit_MAP_CODE1', 'distance_category']}
     )
    )

In [19]:
from humanize import naturalsize
import os
delta_path = './data/parcel.delta'
delta_size = os.path.getsize(delta_path)
print(f"Delta file size: {naturalsize(delta_size)}")

Delta file size: 28.7 kB


In [20]:
from pathlib import Path
folder_size = lambda folder: sum(p.stat().st_size for p in Path(folder).rglob('*'))
print(f"Delta file size: {naturalsize(folder_size('./data/parcel,delta'))}")

Delta file size: 0 Bytes


In [21]:
folder_size = lambda folder: sum(p.stat().st_size for p in Path(folder).rglob('*'))
naturalsize(folder_size('data/parcel.data'))

'0 Bytes'

In [22]:
(
    pl.scan_delta('data/parcel.delta')
    .select('Year')
    .group_by('Year')
    .len()
    .collect()
)

Year,len
str,u32
"""2014""",2116393
"""2005""",1968478
"""2007""",2025481
"""2009""",2088218
"""2015""",2128136
…,…
"""2006""",2007921
"""2011""",2100691
"""2013""",2106911


In [23]:
(new_parcel_row_counts :=
    pl.scan_delta('data/parcel.delta')
    .select('Year')
    .group_by('Year')
    .len()
    .collect()
    .sort('Year')
)

Year,len
str,u32
"""2004""",1982373
"""2005""",1968478
"""2006""",2007921
"""2007""",2025481
"""2008""",2109721
…,…
"""2011""",2100691
"""2012""",2101522
"""2013""",2106911


In [27]:
parcel_row_counts = {
    re.search(r'(\d{4})', path).group(1): (
        pl.scan_csv(path, separator='|', infer_schema=False)
        .select(pl.len())
        .collect()
        .item()
    )
    for path in parcel_paths
}

print(parcel_row_counts)


{'2004': 1982375, '2005': 1968480, '2006': 2007923, '2007': 2025483, '2008': 2109721, '2009': 2088218, '2010': 2097798, '2011': 2100695, '2012': 2101526, '2013': 2106916, '2014': 2116399, '2015': 2128139}


In [28]:
year_pattern = re.compile(r'(\d{4})')
(parcel_row_counts := {
    year_pattern.search(path).group(1): pl.scan_csv(path, separator='|', infer_schema=False).count().collect()
    for path in parcel_paths
})

{'2004': shape: (1, 71)
 ┌────────────┬────────────┬────────────┬────────────┬───┬─────────┬────────┬───────────┬───────────┐
 │ ACRES_DEED ┆ ACRES_POLY ┆ AGPRE_ENRD ┆ AGPRE_EXPD ┆ … ┆ ZIP     ┆ ZIP4   ┆ centroid_ ┆ centroid_ │
 │ ---        ┆ ---        ┆ ---        ┆ ---        ┆   ┆ ---     ┆ ---    ┆ lat       ┆ long      │
 │ u32        ┆ u32        ┆ u32        ┆ u32        ┆   ┆ u32     ┆ u32    ┆ ---       ┆ ---       │
 │            ┆            ┆            ┆            ┆   ┆         ┆        ┆ u32       ┆ u32       │
 ╞════════════╪════════════╪════════════╪════════════╪═══╪═════════╪════════╪═══════════╪═══════════╡
 │ 1982375    ┆ 1982375    ┆ 2291       ┆ 1546       ┆ … ┆ 1799380 ┆ 361165 ┆ 1982375   ┆ 1982373   │
 └────────────┴────────────┴────────────┴────────────┴───┴─────────┴────────┴───────────┴───────────┘,
 '2005': shape: (1, 70)
 ┌────────────┬────────────┬────────────┬────────────┬───┬─────────┬────────┬───────────┬───────────┐
 │ ACRES_DEED ┆ ACRES_POLY ┆ AGPR

In [31]:
original_parcel_row_counts = pl.DataFrame({
    'Year': list(parcel_row_counts.keys()),
    'len': list(parcel_row_counts.values())
})

original_parcel_row_counts


Year,len
str,object
"""2004""","shape: (1, 71) ┌────────────┬────────────┬────────────┬────────────┬───┬─────────┬────────┬───────────┬───────────┐ │ ACRES_DEED ┆ ACRES_POLY ┆ AGPRE_ENRD ┆ AGPRE_EXPD ┆ … ┆ ZIP ┆ ZIP4 ┆ centroid_ ┆ centroid_ │ │ --- ┆ --- ┆ --- ┆ --- ┆ ┆ --- ┆ --- ┆ lat ┆ long │ │ u32 ┆ u32 ┆ u32 ┆ u32 ┆ ┆ u32 ┆ u32 ┆ --- ┆ --- │ │ ┆ ┆ ┆ ┆ ┆ ┆ ┆ u32 ┆ u32 │ ╞════════════╪════════════╪════════════╪════════════╪═══╪═════════╪════════╪═══════════╪═══════════╡ │ 1982375 ┆ 1982375 ┆ 2291 ┆ 1546 ┆ … ┆ 1799380 ┆ 361165 ┆ 1982375 ┆ 1982373 │ └────────────┴────────────┴────────────┴────────────┴───┴─────────┴────────┴───────────┴───────────┘"
"""2005""","shape: (1, 70) ┌────────────┬────────────┬────────────┬────────────┬───┬─────────┬────────┬───────────┬───────────┐ │ ACRES_DEED ┆ ACRES_POLY ┆ AGPRE_ENRD ┆ AGPRE_EXPD ┆ … ┆ ZIP ┆ ZIP4 ┆ centroid_ ┆ centroid_ │ │ --- ┆ --- ┆ --- ┆ --- ┆ ┆ --- ┆ --- ┆ long ┆ lat │ │ u32 ┆ u32 ┆ u32 ┆ u32 ┆ ┆ u32 ┆ u32 ┆ --- ┆ --- │ │ ┆ ┆ ┆ ┆ ┆ ┆ ┆ u32 ┆ u32 │ ╞════════════╪════════════╪════════════╪════════════╪═══╪═════════╪════════╪═══════════╪═══════════╡ │ 1968480 ┆ 1968480 ┆ 2306 ┆ 1571 ┆ … ┆ 1785774 ┆ 561932 ┆ 1968478 ┆ 1968480 │ └────────────┴────────────┴────────────┴────────────┴───┴─────────┴────────┴───────────┴───────────┘"
"""2006""","shape: (1, 70) ┌────────────┬────────────┬────────────┬────────────┬───┬─────────┬────────┬───────────┬───────────┐ │ ACRES_DEED ┆ ACRES_POLY ┆ AGPRE_ENRD ┆ AGPRE_EXPD ┆ … ┆ ZIP ┆ ZIP4 ┆ centroid_ ┆ centroid_ │ │ --- ┆ --- ┆ --- ┆ --- ┆ ┆ --- ┆ --- ┆ long ┆ lat │ │ u32 ┆ u32 ┆ u32 ┆ u32 ┆ ┆ u32 ┆ u32 ┆ --- ┆ --- │ │ ┆ ┆ ┆ ┆ ┆ ┆ ┆ u32 ┆ u32 │ ╞════════════╪════════════╪════════════╪════════════╪═══╪═════════╪════════╪═══════════╪═══════════╡ │ 2007923 ┆ 2007923 ┆ 2348 ┆ 1626 ┆ … ┆ 1857483 ┆ 567434 ┆ 2007921 ┆ 2007923 │ └────────────┴────────────┴────────────┴────────────┴───┴─────────┴────────┴───────────┴───────────┘"
"""2007""","shape: (1, 72) ┌────────────┬────────────┬────────────┬────────────┬───┬─────────┬────────┬───────────┬───────────┐ │ ACRES_DEED ┆ ACRES_POLY ┆ AGPRE_ENRD ┆ AGPRE_EXPD ┆ … ┆ ZIP ┆ ZIP4 ┆ centroid_ ┆ centroid_ │ │ --- ┆ --- ┆ --- ┆ --- ┆ ┆ --- ┆ --- ┆ lat ┆ long │ │ u32 ┆ u32 ┆ u32 ┆ u32 ┆ ┆ u32 ┆ u32 ┆ --- ┆ --- │ │ ┆ ┆ ┆ ┆ ┆ ┆ ┆ u32 ┆ u32 │ ╞════════════╪════════════╪════════════╪════════════╪═══╪═════════╪════════╪═══════════╪═══════════╡ │ 2025483 ┆ 2025483 ┆ 204879 ┆ 204159 ┆ … ┆ 1839159 ┆ 579579 ┆ 2025483 ┆ 2025481 │ └────────────┴────────────┴────────────┴────────────┴───┴─────────┴────────┴───────────┴───────────┘"
"""2008""","shape: (1, 72) ┌────────────┬────────────┬────────────┬────────────┬───┬─────────┬────────┬───────────┬───────────┐ │ ACRES_DEED ┆ ACRES_POLY ┆ AGPRE_ENRD ┆ AGPRE_EXPD ┆ … ┆ ZIP ┆ ZIP4 ┆ centroid_ ┆ centroid_ │ │ --- ┆ --- ┆ --- ┆ --- ┆ ┆ --- ┆ --- ┆ lat ┆ long │ │ u32 ┆ u32 ┆ u32 ┆ u32 ┆ ┆ u32 ┆ u32 ┆ --- ┆ --- │ │ ┆ ┆ ┆ ┆ ┆ ┆ ┆ u32 ┆ u32 │ ╞════════════╪════════════╪════════════╪════════════╪═══╪═════════╪════════╪═══════════╪═══════════╡ │ 2109721 ┆ 2109721 ┆ 1384161 ┆ 1383401 ┆ … ┆ 2007731 ┆ 575387 ┆ 2109721 ┆ 2109721 │ └────────────┴────────────┴────────────┴────────────┴───┴─────────┴────────┴───────────┴───────────┘"
…,…
"""2011""","shape: (1, 70) ┌────────────┬────────────┬────────────┬────────────┬───┬─────────┬────────┬───────────┬───────────┐ │ ACRES_DEED ┆ ACRES_POLY ┆ AGPRE_ENRD ┆ AGPRE_EXPD ┆ … ┆ ZIP ┆ ZIP4 ┆ centroid_ ┆ centroid_ │ │ --- ┆ --- ┆ --- ┆ --- ┆ ┆ --- ┆ --- ┆ long ┆ lat │ │ u32 ┆ u32 ┆ u32 ┆ u32 ┆ ┆ u32 ┆ u32 ┆ --- ┆ --- │ │ ┆ ┆ ┆ ┆ ┆ ┆ ┆ u32 ┆ u32 │ ╞════════════╪════════════╪════════════╪════════════╪═══╪═════════╪════════╪═══════════╪═══════════╡ │ 2100695 ┆ 2100695 ┆ 2774 ┆ 1760 ┆ … ┆ 2004263 ┆ 584390 ┆ 2100691 ┆ 2100695 │ └────────────┴────────────┴────────────┴────────────┴───┴─────────┴────────┴───────────┴───────────┘"
"""2012""","shape: (1, 70) ┌────────────┬────────────┬────────────┬────────────┬───┬─────────┬────────┬───────────┬───────────┐ │ ACRES_DE

In [40]:
# Correctly re-create original_parcel_row_counts to extract scalar row counts
# The 'parcel_row_counts' dictionary contains DataFrames, so we extract the scalar item from each.
original_parcel_row_counts = pl.DataFrame({
    'Year': list(parcel_row_counts.keys()),
    'len': [df.item(0, 0) for df in parcel_row_counts.values()]
}).sort('Year')

parcel_counts_diff = (
    new_parcel_row_counts
    .join(
        original_parcel_row_counts,
        on="Year",
        how="inner",
        suffix="_original",
    )
    .with_columns([
        pl.col("len").cast(pl.Int64, strict=False),
        pl.col("len_original").cast(pl.Int64, strict=False),
        (pl.col("len").cast(pl.Int64, strict=False)
         - pl.col("len_original").cast(pl.Int64, strict=False)
        ).alias("row_count_diff"),
    ])
)

parcel_counts_diff

Year,len,len_original,row_count_diff
str,i64,i64,i64
"""2004""",1982373,1982375,-2
"""2005""",1968478,1968480,-2
"""2006""",2007921,2007923,-2
"""2007""",2025481,2025483,-2
"""2008""",2109721,2109721,0
…,…,…,…
"""2011""",2100691,2100695,-4
"""2012""",2101522,2101526,-4
"""2013""",2106911,2106916,-5


In [42]:
max_diff = parcel_counts_diff['row_count_diff'].max()
min_diff = parcel_counts_diff['row_count_diff'].min()
print(f"Max row count difference: {max_diff}")
print(f"Min row count difference: {min_diff}")

Max row count difference: 0
Min row count difference: -6


In [44]:
row_count_diff_stats = {
    'Max difference': max_diff,
    'Min difference': min_diff
}
row_count_diff_stats

{'Max difference': 0, 'Min difference': -6}

In [45]:
import sys
sys.version_info

sys.version_info(major=3, minor=12, micro=12, releaselevel='final', serial=0)

In [46]:
import polars
polars.__version__

'1.31.0'

In [47]:
import deltalake
deltalake.__version__

'1.2.1'

In [48]:
import pyarrow
pyarrow.__version__

'18.1.0'